In [1]:
import os
import openai
from env_config import Config

openai.api_key = Config.OPENAI_API_KEY

message = ["hi", "i have a cat", "can you suggest a name for her"]
for i in range(3):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": message[i]}
    ],
    n = 1
    )
    print(completion)



{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hello! How can I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1685956083,
  "id": "chatcmpl-7O0Tzjy46PENZxZHEssoUytJhK86m",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 9,
    "total_tokens": 18
  }
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "That's great! Would you like to tell me more about your cat? What is their name, breed, age, personality, etc.?",
        "role": "assistant"
      }
    }
  ],
  "created": 1685956085,
  "id": "chatcmpl-7O0U1gl2qg2YJ1ud6fGJ7ObAzgvKQ",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 12,
    "total_tokens": 40
  }
}
{
  "choices": [
    {
      "finish_reason": "stop",
      "inde